In [97]:
import requests
import json
import re
from bs4 import BeautifulSoup

In [98]:
url = 'https://quotes.toscrape.com/'
next_page = "/page/1/"
quotes_list = []
author_list = []
added_authors = []


In [41]:
while next_page:
    response = requests.get(url + next_page)
    soup = BeautifulSoup(response.text, 'html.parser')

    quotes = soup.find_all('span', class_='text')
    authors = soup.find_all('small', class_='author')
    tags = soup.find_all('div', class_='tags')
    
    for i in range(0, len(quotes)):
        dict_of_quotes = {}
        quote_tags = []

        tagsforquote = tags[i].find_all('a', class_='tag')
        for tagforquote in tagsforquote:
            quote_tags.append(tagforquote.text)

        dict_of_quotes['tags'] = quote_tags
        dict_of_quotes['author'] = authors[i].text
        dict_of_quotes['quote'] = quotes[i].text
        quotes_list.append(dict_of_quotes)

    next_button = soup.find('li', class_='next')
    if next_button:
        next_page = next_button.find('a')['href']
    else:
        next_page = None

    

In [42]:
with open('qoutes.json', 'w', encoding='utf=8') as f:
    json.dump(quotes_list, f, ensure_ascii=False, indent=4)

In [99]:
while next_page:
    response = requests.get(url + next_page)
    soup = BeautifulSoup(response.text, 'html.parser')

    quotes = soup.find_all('span', class_='text')
    authors = soup.find_all('small', class_='author')

    for author_tag in authors:
        dict_of_author = {}

        author_name = author_tag.text.strip()
        if author_name not in added_authors:

            author_link = author_tag.find_next_sibling('a')['href']
            author_url = url + author_link

            response_author = requests.get(author_url)
            author_soup = BeautifulSoup(response_author.text, 'html.parser')

            author_title = author_soup.find('h3', class_='author-title').text.strip() if author_soup.find('h3', class_='author-title') else 'N/A'
            born_date = author_soup.find('span', class_='author-born-date').text.strip() if author_soup.find('span', class_='author-born-date') else 'N/A'
            born_location = author_soup.find('span', class_='author-born-location').text.strip() if author_soup.find('span', class_='author-born-location') else 'N/A'
            description = author_soup.find('div', class_='author-description').text.strip() if author_soup.find('div', class_='author-description') else 'N/A'

            dict_of_author['fullname'] = author_title
            dict_of_author['born_date'] = born_date
            dict_of_author['born_location'] = born_location
            dict_of_author['description'] = description

            author_list.append(dict_of_author)
            added_authors.append(author_name)
                
    next_button = soup.find('li', class_='next')
    if next_button:
        next_page = next_button.find('a')['href']
    else:
        next_page = None

In [100]:
with open('authors.json', 'w', encoding='utf=8') as f:
    json.dump(author_list, f, ensure_ascii=False, indent=4)